In [1]:
import torch
import pandas as pd
import gzip
from torch_geometric.data import HeteroData
from torch_geometric.utils import add_self_loops

In [2]:
# 解压并读取数据
def read_data(file_path):
    with gzip.open(file_path, 'rt') as f:
        df = pd.read_csv(f, sep='\t', header=None, names=['source', 'relation', 'target'])
    return df

# 创建异构图并初始化特征和关系标签
def create_hetero_graph():
    data = HeteroData()

    # 定义不同的文件路径
    file_paths = {
        'Drug-Protein': 'drug_protein.txt.gz',
        'DDI': 'ddi.txt.gz',
        'Protein-Pathway': 'protein_pathway.txt.gz',
        'Drug-Pathway': 'drug_pathway.txt.gz',
        'Protein-Disease': 'protein_disease.txt.gz',
        'Drug-Disease': 'drug_disease.txt.gz',
        'PPI': 'ppi.txt.gz'
    }
    
    # 用于存储所有的节点和边
    node_types = {'Protein', 'Drug', 'Disease', 'Pathway'}
    edge_types = [
        ('Drug', 'Drug-Protein', 'Protein'),
        ('Drug', 'DDI', 'Drug'),
        ('Protein', 'Protein-Pathway', 'Pathway'),
        ('Drug', 'Drug-Pathway', 'Pathway'),
        ('Protein', 'Protein-Disease', 'Disease'),
        ('Drug', 'Drug-Disease', 'Disease'),
        ('Protein', 'PPI', 'Protein')
    ]
    
    # 为每个节点类型创建 ID 映射
    node_id = {ntype: {} for ntype in node_types}
    node_count = {ntype: 0 for ntype in node_types}

    # 为每种边类型分配一个唯一的标签（0 到 6）
    relation_labels = {edge_types[i]: i for i in range(len(edge_types))}
    
    # 读取文件并填充图数据
    for edge_type in edge_types:
        file_name = file_paths[edge_type[1]]
        print(f"Processing edge type: {edge_type}, file: {file_name}")
        dir_path = '/data/zhaojingtong/pharmrgdata/'
        df = read_data(dir_path + file_name)
        src_type, relation, tgt_type = edge_type

        if src_type not in node_id:
            node_id[src_type] = {}
        if tgt_type not in node_id:
            node_id[tgt_type] = {}

        # 为源节点和目标节点创建 ID
        src_ids = []
        tgt_ids = []

        for _, row in df.iterrows():
            src = row['source']
            tgt = row['target']

            if src not in node_id[src_type]:
                node_id[src_type][src] = node_count[src_type]
                node_count[src_type] += 1
            if tgt not in node_id[tgt_type]:
                node_id[tgt_type][tgt] = node_count[tgt_type]
                node_count[tgt_type] += 1

            src_ids.append(node_id[src_type][src])
            tgt_ids.append(node_id[tgt_type][tgt])

        # 转换为 PyG 格式
        edge_index = torch.tensor([src_ids, tgt_ids], dtype=torch.long)
        data[edge_type].edge_index = edge_index

        # 初始化边特征，128维度随机向量
        num_edges = edge_index.size(1)
        data[edge_type].edge_attr = torch.randn(num_edges, 128)

        # 为每种边类型分配标签，使用 relation_labels
        data[edge_type].edge_label = torch.full((num_edges,), relation_labels[edge_type], dtype=torch.long)

    # 创建节点特征，初始化为128维随机向量
    for ntype, id_map in node_id.items():
        num_nodes = len(id_map)
        data[ntype].num_nodes = num_nodes
        data[ntype].x = torch.randn(num_nodes, 128)  # 128维度随机特征

    return data

# 生成异构图并初始化特征和标签
hetero_data = create_hetero_graph()

# 检查生成的异构图数据
print(hetero_data)


Processing edge type: ('Drug', 'Drug-Protein', 'Protein'), file: drug_protein.txt.gz
Processing edge type: ('Drug', 'DDI', 'Drug'), file: ddi.txt.gz



KeyboardInterrupt



In [4]:
import torch
save_path = '/data/zhaojingtong/pharmrgdata/hetero_graph.pt'
# 假设 hetero_data 是你构建好的异构图对象
# torch.save(hetero_data, save_path)

In [5]:
hetero_data = torch.load(save_path)
print(hetero_data)

HeteroData(
  Drug={
    num_nodes=9279,
    x=[9279, 128]
  },
  Disease={
    num_nodes=5819,
    x=[5819, 128]
  },
  Pathway={
    num_nodes=15771,
    x=[15771, 128]
  },
  Protein={
    num_nodes=62295,
    x=[62295, 128]
  },
  (Drug, Drug-Protein, Protein)={
    edge_index=[2, 25994],
    edge_attr=[25994, 128],
    edge_label=[25994]
  },
  (Drug, DDI, Drug)={
    edge_index=[2, 1419538],
    edge_attr=[1419538, 128],
    edge_label=[1419538]
  },
  (Protein, Protein-Pathway, Pathway)={
    edge_index=[2, 281950],
    edge_attr=[281950, 128],
    edge_label=[281950]
  },
  (Drug, Drug-Pathway, Pathway)={
    edge_index=[2, 5114],
    edge_attr=[5114, 128],
    edge_label=[5114]
  },
  (Protein, Protein-Disease, Disease)={
    edge_index=[2, 116699],
    edge_attr=[116699, 128],
    edge_label=[116699]
  },
  (Drug, Drug-Disease, Disease)={
    edge_index=[2, 61726],
    edge_attr=[61726, 128],
    edge_label=[61726]
  },
  (Protein, PPI, Protein)={
    edge_index=[2, 131406],


In [10]:
import torch

# 初始化每种类型的节点特征为128维度随机向量
def initialize_node_edge_features(hetero_data):
    feature_dim = 1024  # 设定特征维度为128

    # 为每种节点类型初始化特征
    for node_type in hetero_data.node_types:
        num_nodes = hetero_data[node_type].num_nodes
        # 生成随机特征向量，形状为 [num_nodes, 128]
        hetero_data[node_type].x = torch.randn(num_nodes, feature_dim)
        
    for edge_type in hetero_data.edge_types:
        num_edges = hetero_data[edge_type].edge_index.size(1)  # 获取该类型的边数量
        # 生成随机特征向量，形状为 [num_edges, 128]
        hetero_data[edge_type].edge_attr = torch.randn(num_edges, feature_dim)
    return hetero_data

# 假设 hetero_data 是你构建好的异构图
hetero_data = initialize_node_edge_features(hetero_data)

# 检查每个节点类型的特征维度是否正确
print(hetero_data)

HeteroData(
  Drug-Disease=[2, 61726],
  Protein-Pathway=[2, 281950],
  Protein-Disease=[2, 116699],
  DDI=[2, 1419538],
  Drug-Pathway=[2, 5114],
  PPI=[2, 131406],
  Drug-Protein=[2, 25994],
  Drug={
    num_nodes=9279,
    x=[9279, 1024]
  },
  Disease={
    num_nodes=5819,
    x=[5819, 1024]
  },
  Pathway={
    num_nodes=15771,
    x=[15771, 1024]
  },
  Protein={
    num_nodes=62295,
    x=[62295, 1024]
  }
)


In [11]:
torch.save(hetero_data, save_path)

In [26]:
hetero_data.edge_label_dict


{('Drug', 'Drug-Protein', 'Protein'): tensor([0, 0, 0,  ..., 0, 0, 0]),
 ('Drug', 'DDI', 'Drug'): tensor([1, 1, 1,  ..., 1, 1, 1]),
 ('Protein', 'Protein-Pathway', 'Pathway'): tensor([2, 2, 2,  ..., 2, 2, 2]),
 ('Drug', 'Drug-Pathway', 'Pathway'): tensor([3, 3, 3,  ..., 3, 3, 3]),
 ('Protein', 'Protein-Disease', 'Disease'): tensor([4, 4, 4,  ..., 4, 4, 4]),
 ('Drug', 'Drug-Disease', 'Disease'): tensor([5, 5, 5,  ..., 5, 5, 5]),
 ('Protein', 'PPI', 'Protein'): tensor([6, 6, 6,  ..., 6, 6, 6])}